<a href="https://colab.research.google.com/github/DockyJ/social_network_calls_Brazil/blob/main/sn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, csv, collections, datetime
import networkx as nx
from itertools import islice
from math import radians, cos, sin, asin, sqrt, log1p
import unicodedata, gc
import seaborn as sns
%matplotlib inline

from scipy import stats
from scipy.stats import normaltest

In [ ]:
# Load Google shared folder
from google.colab import drive
drive.mount('/content/drive')

# Common Utility Functions

In [ ]:
def normalize_string(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn').lower()

In [ ]:
def read_ana(file_path_ana):
    """
    Read the antenna locations file and return a dictionary of antenna locations.
    """
    # Load antenna locations
    antenna_locations = {}

    with open(file_path_ana, 'r') as file:
        reader = csv.reader(file, delimiter=';')
        next(reader)  # Skip the header
        for row in reader:
            cell_id, lat, lon = row[0], float(row[3]), float(row[4])
            antenna_locations[cell_id] = (lat, lon)
    return antenna_locations

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return c * 6371 # Radius of earth in kilometers

In [ ]:
def home_location_calculation(calls_users_time_filter, significant_users):
    """
    Calculate the home location based on the filtered records.
    """
    # Initialize variables
    home_locations = {}

    for user, locations in calls_users_time_filter.items():
        if user in significant_users:
            most_frequent_cell, max_calls = max(locations.items(), key=lambda item: item[1])
            total_calls = sum(locations.values())
            if max_calls >= total_calls * 0.5:
                home_locations[user] = (most_frequent_cell, max_calls)
    return home_locations

In [ ]:
def write_data_to_csv(df, city, city_data, output_suffix):
    """
    Write the data to a CSV file.
    """

    # Define the main directory and subdirectory paths
    main_directory = '/content/drive/MyDrive/network_radius_city_size'
    subdirectory = os.path.join(main_directory, output_suffix)

    # Create the subdirectory if it does not exist
    os.makedirs(subdirectory, exist_ok=True)

    # Construct the full file path
    file_path = os.path.join(subdirectory, f'network_radius_{output_suffix}.csv')

    # Determine the file mode based on header_written
    file_mode = 'w' if not header_written else 'a'

    selected_row = df[df['City_lower'] == normalize_string(city)].iloc[0]
    data_to_write = [
        city,
        *city_data,
        selected_row['Population 2010'],
        selected_row['DENS HAB KM2'],
        selected_row['Income 2010'],
        selected_row['GDP 2010'],
        selected_row['GDP 2017'],
        selected_row['IDHM 2010'],
        selected_row['Comp Total Ruas'],
        selected_row['Number Of Deaths By Traffic Accident'],
        selected_row['surfaceOfAdministrativeArea km2']
    ]

    with open(file_path, file_mode, newline='') as csvfile:
        writer = csv.writer(csvfile)
        if not header_written:
                writer.writerow([
                    'City', 'Network Radius',
                    'Total Users',
                    'Users after Filter 1', 'Percentage Filtered 1 Users',
                    'Significant Users', 'Percentage Significant Users',
                    'Final Users', 'Percentage Final Users',
                    'Total Calls Weight', 'Final Calls Weight',
                    'Percentage Final Calls Weight',
                    'Records Callee Missed Home Location',
                    'Zero Radius Users', 'Percentage Zero Radius Users',
                    'Spam Calls', 'Percentage Spam Calls',
                    'Number of Antennas',

                    'Population 2010', 'DENS HAB KM2',
                    'Income 2010', 'GDP 2010',
                    'GDP 2017', 'IDHM 2010',
                    'Comp Total Ruas', 'Number Of Deaths By Traffic Accident',
                    'surfaceOfAdministrativeArea km2'
                ])
        writer.writerow(data_to_write)
    print(f"Finish writing radius of {city}")

In [ ]:
def network_radius_calculation(calls_between_users, home_locations, antenna_locations, is_log=False):
    """
    Calculate the network radius based on the calls between users and their home locations
    """
    # Initialize
    total_users = 0
    total_distance_weighted = 0.0
    total_weight_final = 0.0
    callee_miss_home = 0

    # Calculate social network distances
    social_network = {}
    for caller, callees in calls_between_users.items():
        if caller in home_locations:
            caller_home = home_locations[caller][0]
            if caller_home in antenna_locations:
                lat1, lon1 = antenna_locations[caller_home]
                for callee, call_weight in callees.items():
                    if callee in home_locations:
                        callee_home = home_locations[callee][0]
                        if callee_home in antenna_locations:
                            lat2, lon2 = antenna_locations[callee_home]
                            distance = haversine(lat1, lon1, lat2, lon2)

                            # Apply log transform to call_weight if is_log is True
                            if is_log:
                                call_weight = log1p(call_weight)

                            # Update social network distances
                            if caller not in social_network:
                                social_network[caller] = []
                            social_network[caller].append(distance * call_weight)
                            total_weight_final += call_weight
                    else:
                        callee_miss_home += 1


    # Calculate Final Users
    total_users_final = len(social_network)

    # Compute network radius
    total_distance_weighted = sum([sum(distances) for distances in social_network.values()])
    network_radius = total_distance_weighted / total_users_final if total_users_final else 0

    del social_network
    gc.collect()

    return network_radius, total_weight_final, total_users_final, callee_miss_home

In [ ]:
def network_radius_calculation_individual(calls_between_users, home_locations, antenna_locations, city, output_suffix, is_log=False, non_zero=False):
    """
    Calculate the network radius based on individual callers' network radii and then average.

    Args:
        calls_between_users: Dictionary with callers and their callees.
        home_locations: Dictionary mapping users to their home location (cell).
        antenna_locations: Dictionary mapping cell IDs to (latitude, longitude).
        city: The name of the city for saving plots.
        is_log: Whether to apply a log transform to the call weights.

    Returns:
        Average network radius for the city, total weight, number of final users, and number of users with missing home locations.
    """
    # Initialize variables
    individual_radii = []
    total_weight_final = 0.0
    callee_miss_home_record = 0
    zero_radius_count = 0

    # FOR Distribution analysis
    individual_radii_nozero=[]

    # Calculate network radius for each caller
    for caller, callees in calls_between_users.items():
        if caller in home_locations:
            caller_home = home_locations[caller][0]
            if caller_home in antenna_locations:
                lat1, lon1 = antenna_locations[caller_home]
                distances = []
                total_weight = 0
                distances_nozero = []

                for callee, call_weight in callees.items():
                    if callee in home_locations:
                        callee_home = home_locations[callee][0]
                        if callee_home in antenna_locations:
                            lat2, lon2 = antenna_locations[callee_home]
                            distance = haversine(lat1, lon1, lat2, lon2)

                            if distance == 0:
                                # distance = 0.5

                                # FOR Distribution analysis
                                distance_nozero = 0.5
                            else:
                                distance_nozero = distance

                            # Calculate weighted distance
                            distances.append(distance * call_weight)

                            # FOR Distribution analysis
                            distances_nozero.append(distance_nozero * call_weight)

                            total_weight += call_weight
                    else:
                        # callee_miss_home contains duplicated callees as it is actually call records of missing callees
                        callee_miss_home_record += 1

                # Calculate the individual's network radius
                if total_weight > 0:
                    individual_radius = sum(distances) / len(callees)

                    # FOR Distribution analysis
                    individual_radius_nozero = sum(distances_nozero) / len(callees)
                    individual_radii_nozero.append(individual_radius_nozero)

                    if is_log:
                        individual_radius = np.log1p(individual_radius)

                    if individual_radius == 0:
                        zero_radius_count += 1
                        if not non_zero:  # Only append if non_zero is False
                            individual_radii.append(individual_radius)
                    else:
                        individual_radii.append(individual_radius)
                    total_weight_final += total_weight

    # Calculate the average network radius
    network_radius = np.mean(individual_radii) if individual_radii else 0
    total_users_final = len(individual_radii)

    # Plot the distribution of individual network radii
    plot_network_radius_distribution(individual_radii, individual_radii_nozero, city, output_suffix)

    return network_radius, total_weight_final, total_users_final, callee_miss_home_record, zero_radius_count

In [ ]:
def plot_network_radius_distribution(individual_radii, individual_radii_nozero, city, output_suffix):
    """
    Plot the distribution of individual network radii and save the plot.

    Args:
        individual_radii: List of individual network radii.
        individual_radii_nozero: List of individual network radii excluding zero values.
        city: The name of the city for saving plots.
        output_suffix: Suffix for the output file names.
    """
    main_directory = '/content/drive/MyDrive/network_radius_city_size'
    subdirectory_1 = os.path.join(main_directory, city)
    os.makedirs(subdirectory_1, exist_ok=True)

    subdirectory = os.path.join(subdirectory_1, output_suffix)
    os.makedirs(subdirectory, exist_ok=True)

    # Construct the file paths
    hist_file_path = os.path.join(subdirectory, f'network_radius_distribution_{city}_{output_suffix}.png')
    log_hist_file_path = os.path.join(subdirectory, f'log_network_radius_distribution_{city}_{output_suffix}.png')
    qq_file_path = os.path.join(subdirectory, f'qq_plot_{city}_{output_suffix}.png')

    hist_file_path_nozero = os.path.join(subdirectory, f'network_radius_distribution_nozero_{city}_{output_suffix}.png')
    log_hist_file_path_nozero = os.path.join(subdirectory, f'log_network_radius_distribution_nozero_{city}_{output_suffix}.png')
    qq_file_path_nozero = os.path.join(subdirectory, f'qq_plot_nozero_{city}_{output_suffix}.png')

    sqrt_hist_file_path = os.path.join(subdirectory, f'sqrt_network_radius_distribution_{city}_{output_suffix}.png')
    sqrt_hist_file_path_nozero = os.path.join(subdirectory, f'sqrt_network_radius_distribution_nozero_{city}_{output_suffix}.png')
    sqrt_qq_file_path = os.path.join(subdirectory, f'sqrt_qq_plot_{city}_{output_suffix}.png')
    sqrt_qq_file_path_nozero = os.path.join(subdirectory, f'sqrt_qq_plot_nozero_{city}_{output_suffix}.png')

    # Plot the distribution for non-zero values
    plt.figure(figsize=(10, 6))
    plt.hist(individual_radii_nozero, bins=30, color='blue', alpha=0.7)
    plt.title(f'Network Radius Distribution for {city} (Non-Zero Values)')
    plt.xlabel('Network Radius (km)')
    plt.ylabel('Frequency')
    plt.grid(True)

    # Calculate and display statistical summary
    desc_stats = pd.Series(individual_radii_nozero).describe()
    stats_text = '\n'.join([f'{key}: {value:.2f}' for key, value in desc_stats.items()])
    plt.figtext(0.35, 0.5, f'Statistical Summary:\n{stats_text}', fontsize=10, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

    # Save the plot
    plt.savefig(hist_file_path_nozero)
    plt.close()

    # Plot the log-transformed distribution for non-zero values
    log_radii_nozero = np.log1p(individual_radii_nozero)
    plt.figure(figsize=(10, 6))
    plt.hist(log_radii_nozero, bins=30, color='green', alpha=0.7)
    plt.title(f'Log-Transformed Network Radius Distribution for {city} (Non-Zero Values)')
    plt.xlabel('Log(Network Radius + 1)')
    plt.ylabel('Frequency')
    plt.grid(True)

    # Calculate and display statistical summary for log data
    log_desc_stats = pd.Series(log_radii_nozero).describe()
    log_stats_text = '\n'.join([f'{key}: {value:.2f}' for key, value in log_desc_stats.items()])
    plt.figtext(0.35, 0.5, f'Log Statistical Summary:\n{log_stats_text}', fontsize=10, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

    # Save the plot
    plt.savefig(log_hist_file_path_nozero)
    plt.close()

    # Plot Q-Q plot for non-zero values
    plt.figure(figsize=(6, 6))
    stats.probplot(log_radii_nozero, dist="norm", plot=plt)
    plt.title(f'Q-Q Plot of Log-Transformed Network Radius for {city} (Non-Zero Values)')

    # Save the Q-Q plot
    plt.savefig(qq_file_path_nozero)
    plt.close()

    # Plot the square root-transformed distribution for non-zero values
    sqrt_radii_nozero = np.sqrt(individual_radii_nozero)
    plt.figure(figsize=(10, 6))
    plt.hist(sqrt_radii_nozero, bins=30, color='purple', alpha=0.7)
    plt.title(f'Square Root-Transformed Network Radius Distribution for {city} (Non-Zero Values)')
    plt.xlabel('Square Root(Network Radius)')
    plt.ylabel('Frequency')
    plt.grid(True)

    # Calculate and display statistical summary for sqrt data
    sqrt_desc_stats = pd.Series(sqrt_radii_nozero).describe()
    sqrt_stats_text = '\n'.join([f'{key}: {value:.2f}' for key, value in sqrt_desc_stats.items()])
    plt.figtext(0.35, 0.5, f'Sqrt Statistical Summary:\n{sqrt_stats_text}', fontsize=10, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

    # Save the plot
    plt.savefig(sqrt_hist_file_path_nozero)
    plt.close()

    # Plot Q-Q plot for sqrt-transformed distribution
    plt.figure(figsize=(6, 6))
    stats.probplot(sqrt_radii_nozero, dist="norm", plot=plt)
    plt.title(f'Q-Q Plot of Sqrt-Transformed Network Radius for {city} (Non-Zero Values)')

    # Save the Q-Q plot
    plt.savefig(sqrt_qq_file_path_nozero)
    plt.close()

    # Plot the distribution
    plt.figure(figsize=(10, 6))
    plt.hist(individual_radii, bins=30, color='blue', alpha=0.7)
    plt.title(f'Network Radius Distribution for {city}')
    plt.xlabel('Network Radius (km)')
    plt.ylabel('Frequency')
    plt.grid(True)

    # Calculate and display statistical summary
    desc_stats = pd.Series(individual_radii).describe()
    stats_text = '\n'.join([f'{key}: {value:.2f}' for key, value in desc_stats.items()])
    plt.figtext(0.35, 0.5, f'Statistical Summary:\n{stats_text}', fontsize=10, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

    # Save the plot
    plt.savefig(hist_file_path)
    plt.close()

    # Plot the log-transformed distribution
    log_radii = np.log1p(individual_radii)
    plt.figure(figsize=(10, 6))
    plt.hist(log_radii, bins=30, color='green', alpha=0.7)
    plt.title(f'Log-Transformed Network Radius Distribution for {city}')
    plt.xlabel('Log(Network Radius + 1)')
    plt.ylabel('Frequency')
    plt.grid(True)

    # Calculate and display statistical summary for log data
    log_desc_stats = pd.Series(log_radii).describe()
    log_stats_text = '\n'.join([f'{key}: {value:.2f}' for key, value in log_desc_stats.items()])
    plt.figtext(0.35, 0.5, f'Log Statistical Summary:\n{log_stats_text}', fontsize=10, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

    # Save the plot
    plt.savefig(log_hist_file_path)
    plt.close()

    # Plot Q-Q plot
    plt.figure(figsize=(6, 6))
    stats.probplot(log_radii, dist="norm", plot=plt)
    plt.title(f'Q-Q Plot of Log-Transformed Network Radius for {city}')

    # Save the Q-Q plot
    plt.savefig(qq_file_path)
    plt.close()

    # Plot the square root-transformed distribution
    sqrt_radii = np.sqrt(individual_radii)
    plt.figure(figsize=(10, 6))
    plt.hist(sqrt_radii, bins=30, color='purple', alpha=0.7)
    plt.title(f'Square Root-Transformed Network Radius Distribution for {city}')
    plt.xlabel('Square Root(Network Radius)')
    plt.ylabel('Frequency')
    plt.grid(True)

    # Calculate and display statistical summary for sqrt data
    sqrt_desc_stats = pd.Series(sqrt_radii).describe()
    sqrt_stats_text = '\n'.join([f'{key}: {value:.2f}' for key, value in sqrt_desc_stats.items()])
    plt.figtext(0.35, 0.5, f'Sqrt Statistical Summary:\n{sqrt_stats_text}', fontsize=10, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

    # Save the plot
    plt.savefig(sqrt_hist_file_path)
    plt.close()

    # Plot Q-Q plot for sqrt-transformed distribution
    plt.figure(figsize=(6, 6))
    stats.probplot(sqrt_radii, dist="norm", plot=plt)
    plt.title(f'Q-Q Plot of Sqrt-Transformed Network Radius for {city}')

    # Save the Q-Q plot
    plt.savefig(sqrt_qq_file_path)
    plt.close()


In [ ]:
def convert_dict_to_df_count(calls_dict):
    # Prepare empty lists to fill with data
    data = {'caller_id': [], 'id2': [], 'call_weight': []}

    for caller_id, cells in calls_dict.items():
        for id2, weight in cells.items():
            data['caller_id'].append(caller_id)
            data['id2'].append(id2)
            data['call_weight'].append(weight)

    # Create DataFrame from the collected data
    df = pd.DataFrame(data)

    # Convert data types to save memory
    df['caller_id']  = df['caller_id'].astype('category')   # Reduces memory by using categorical data type for repetitive strings
    df['id2']   = df['id2'].astype('category')        # Similar to caller_id, using category data type for cell ids
    df['call_weight'] = df['call_weight'].astype('float32')   # Using float32 for call weights to accommodate decimal values

    # Aggregate call weights by caller_id
    df_summed = df.groupby('caller_id')['call_weight'].sum().reset_index()
    df_summed.columns = ['caller_id', 'total_call_weight']

    return df_summed

In [ ]:
def convert_dict_to_df_dura(calls_dict):
    # Prepare empty lists to fill with data
    data = {'caller_id': [], 'callee_id': [], 'total_duration': []}

    # Aggregate durations by caller_id to callee_id
    for caller_id, callees in calls_dict.items():
        for callee_id, duration in callees.items():
            data['caller_id'].append(caller_id)
            data['callee_id'].append(callee_id)
            data['total_duration'].append(duration)

    # Create DataFrame from the collected data
    df = pd.DataFrame(data)

    # Convert data types to save memory and reflect correct types
    df['caller_id'] = df['caller_id'].astype('category')  # Reduces memory for repetitive strings
    df['callee_id'] = df['callee_id'].astype('category')  # Same as caller_id
    df['total_duration'] = df['total_duration'].astype('float32')  # Ensures duration is in float format

    return df

## Utility Functions for using counts as the weight

In [ ]:
def read_cdr_count(file_path_cdr):
    """
    Read the CDR file and return a dictionary of user call records,
    significant user call records based on time conditions,
    and calls between users.
    """

    # Dictionary to hold significant user call records based on time conditions to calculate the home location
    calls_users_time_filter = {}
    # Dictionary to store the calls between users' ids
    calls_between_users = {}
    # Spam call possibility
    dura_spam = 0


    # Read the data file line by line
    with open(file_path_cdr, 'r') as file:
        for line in file:
            # Split each line into parts
            parts = line.strip().split(';')

            duration = float(parts[2])
            if duration <= 0.1:
                dura_spam += 1
                continue

            # Extract the columns of interest: Date, Time, Caller ID, Cell ID
            date_time_str = parts[0] + ' ' + parts[1]
            caller_id = parts[4]
            callee_id = parts[6]
            cell_id = parts[7]
            # Parse date and time
            date_time = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
            weekday = date_time.weekday()
            hour = date_time.hour



            # Store calls between users
            if caller_id not in calls_between_users:
                calls_between_users[caller_id] = {}
            else:
                if callee_id not in calls_between_users[caller_id]:
                    calls_between_users[caller_id][callee_id] = 1
                else:
                    calls_between_users[caller_id][callee_id] += 1

            # Filter based on time conditions
            if (weekday < 5 and (hour >= 19 or hour <= 6)) or (weekday >= 5):
                if caller_id not in calls_users_time_filter:
                    calls_users_time_filter[caller_id] = {}

                # Count calls per location for significant times
                if cell_id not in calls_users_time_filter[caller_id]:
                    calls_users_time_filter[caller_id][cell_id] = 1
                else:
                    calls_users_time_filter[caller_id][cell_id] += 1
    return calls_users_time_filter, calls_between_users, dura_spam

In [ ]:
def significant_users_calculation(calls_users_time_filter):
    """
    Calculate the significant users based on the filtered records.
    """
    # Initialize variables
    significant_users = set()

    # Determine the significant users based on the filtered records
    for user, locations in calls_users_time_filter.items():
        # if user not in excluded_users and 5 <= sum(locations.values()) <= 200:
        #     significant_users.add(user)
        if 5 <= sum(locations.values()) <= 200:
            significant_users.add(user)
    return significant_users

## Utility Functions for using duration as the weight

In [ ]:
def read_cdr_duration(file_path_cdr):
    """
    Read the CDR file and return a dictionary of user call records,
    significant user call records based on time conditions,
    and calls between users.
    """

    # Dictionary for holding significant user call records based on time conditions to calculate the home location
    calls_users_time_filter = {}
    # Dictionary for accumulating duration from callers to the same callees to calculate the social network radius
    calls_users_duration = {}
    # Spam possibility
    dura_spam = 0

    # Read the data file line by line
    with open(file_path_cdr, 'r') as file:
        for line in file:
            # Split lines into parts
            parts = line.strip().split(';')

            # Filter out spam calls
            duration = float(parts[2])
            if duration <= 0.1:
                dura_spam += 1
                continue

            # Extract columns of interest: Date, Time, Duration, call id, cell id
            data_time_str = parts[0] + ' ' + parts[1]
            caller_id = parts[4]
            callee_id = parts[6]
            cell_id = parts[7]

            # Parse date and time
            date_time = datetime.datetime.strptime(data_time_str, '%Y-%m-%d %H:%M:%S')
            weekday = date_time.weekday()
            hour = date_time.hour

            # Filter based on time conditions for significant users determination
            if (weekday < 5 and (hour >= 19 or hour <= 6)) or (weekday >= 5):
                if caller_id not in calls_users_time_filter:
                    calls_users_time_filter[caller_id] = {}

                # Count calls per location for significant times
                if cell_id not in calls_users_time_filter[caller_id]:
                    calls_users_time_filter[caller_id][cell_id] = 1
                else:
                    calls_users_time_filter[caller_id][cell_id] += 1

            # Accumulate duration for callers to callees
            if caller_id not in calls_users_duration:
                calls_users_duration[caller_id] = {}
            else:
                if callee_id not in calls_users_duration[caller_id]:
                    calls_users_duration[caller_id][callee_id] = float(duration)
                else:
                    calls_users_duration[caller_id][callee_id] += float(duration)

    return calls_users_time_filter, calls_users_duration, dura_spam


## Utility Functions for using actual location

In [ ]:
def read_cdr_count_cell(file_path_cdr):
    """
    Read the CDR file and return a dictionary of user call records,
    significant user call records based on time conditions,
    and calls between users.
    """

    # Dictionary to hold significant user call records based on time conditions to calculate the home location
    calls_users_time_filter = {}
    # List to store the call record
    calls_between_users = []
    # Spam call possibility
    dura_spam = 0


    # Read the data file line by line
    with open(file_path_cdr, 'r') as file:
        for line in file:
            # Split each line into parts
            parts = line.strip().split(';')

            duration = float(parts[2])
            if duration <= 0.1:
                dura_spam += 1
                continue

            # Extract the columns of interest: Date, Time, Caller ID, Cell ID
            date_time_str = parts[0] + ' ' + parts[1]
            caller_id = parts[4]
            callee_id = parts[6]
            cell_id = parts[7]

            # Parse date and time
            date_time = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
            weekday = date_time.weekday()
            hour = date_time.hour

            # Store calls between users
            calls_between_users.append({
                'caller_id': caller_id,
                'callee_id': callee_id,
                'cell_id': cell_id,
            })

            # Filter based on time conditions
            if (weekday < 5 and (hour >= 19 or hour <= 6)) or (weekday >= 5):
                if caller_id not in calls_users_time_filter:
                    calls_users_time_filter[caller_id] = {}

                # Count calls per location for significant times
                if cell_id not in calls_users_time_filter[caller_id]:
                    calls_users_time_filter[caller_id][cell_id] = 1
                else:
                    calls_users_time_filter[caller_id][cell_id] += 1
    return calls_users_time_filter, calls_between_users, dura_spam

In [ ]:
def network_radius_calculation_cell(calls_between_users, home_locations, antenna_locations, city, output_suffix, is_log=False, non_zero=False):
    """
    Calculate the network radius based on the calls between users using the caller's cell location
    and the callee's home location, then average. Individually calculate the network radius for each caller.

    Args:
        calls_between_users: List of call records with caller_id, callee_id, and cell_id.
        home_locations: Dictionary mapping users to their home location (cell).
        antenna_locations: Dictionary mapping cell IDs to (latitude, longitude).
        city: The name of the city for saving plots.
        is_log: Whether to apply a log transform to the call weights.

    Returns:
        Average network radius for the city, total weight, number of final users, and number of users with missing home locations.
    """
    # Initialize variables
    individual_radii = []
    total_weight_final = 0.0
    callee_miss_home_record = 0
    zero_radius_count = 0

    # Create a dictionary to hold distances and weights per caller
    caller_distances = {}

    # FOR Distribution analysis
    individual_radii_nozero = []

    # Calculate network radius for each caller
    for call_record in calls_between_users:
        caller_id = call_record['caller_id']
        callee_id = call_record['callee_id']
        caller_cell_id = call_record['cell_id']

        # Get caller's location using the cell ID for each call record
        caller_location = antenna_locations.get(caller_cell_id, (np.nan, np.nan))

        # Get callee's home location if available
        callee_homes = home_locations.get(callee_id, None)

        if callee_homes is not None:
            callee_location = antenna_locations.get(callee_homes[0], (np.nan, np.nan))

            # Proceed if both locations are valid
            if not np.isnan(caller_location[0]) and not np.isnan(caller_location[1]) and \
               not np.isnan(callee_location[0]) and not np.isnan(callee_location[1]):

                distance = haversine(*caller_location, *callee_location)

                if distance == 0:
                    # distance = 0.5

                    # FOR Distribution analysis
                    distance_nozero = 0.5
                else:
                    distance_nozero = distance

                # Initialize call weight
                call_weight = 1  # Assuming each call record is weighted equally

                # Add distance and weight to caller's record
                if caller_id not in caller_distances:
                    caller_distances[caller_id] = {'distances': [], 'weights': [], 'distances_nozero': []}
                caller_distances[caller_id]['distances'].append(distance * call_weight)
                caller_distances[caller_id]['weights'].append(call_weight)

                # FOR Distribution analysis
                caller_distances[caller_id]['distances_nozero'].append(distance_nozero * call_weight)
        else:
            callee_miss_home_record += 1

    # Calculate the network radius for each caller
    for caller_id, data in caller_distances.items():
        total_weight = sum(data['weights'])
        if total_weight > 0:
            individual_radius = sum(data['distances']) / total_weight

            # FOR Distribution analysis
            individual_radius_nozero = sum(data['distances_nozero']) / total_weight
            individual_radii_nozero.append(individual_radius_nozero)

            if is_log:
                individual_radius = np.log1p(individual_radius)

            if individual_radius == 0:
                zero_radius_count += 1
                if not non_zero:  # Only append if non_zero is False
                    individual_radii.append(individual_radius)
            else:
                individual_radii.append(individual_radius)
            total_weight_final += total_weight

    # Calculate the average network radius
    network_radius = np.mean(individual_radii) if individual_radii else 0
    total_users_final = len(individual_radii)

    # Plot the distribution of individual network radii
    plot_network_radius_distribution(individual_radii, individual_radii_nozero, city, output_suffix)

    return network_radius, total_weight_final, total_users_final, callee_miss_home_record, zero_radius_count

## Utility Functions for using duration as the weight and actual location

In [ ]:
def read_cdr_count_cell_dura(file_path_cdr):
    """
    Read the CDR file and return a dictionary of user call records,
    significant user call records based on time conditions,
    and calls between users.
    """

    # Dictionary to hold significant user call records based on time conditions to calculate the home location
    calls_users_time_filter = {}
    # List to store the call record
    calls_between_users = []
    # Spam call possibility
    dura_spam = 0

    # Read the data file line by line
    with open(file_path_cdr, 'r') as file:
        for line in file:
            # Split each line into parts
            parts = line.strip().split(';')

            duration = float(parts[2])
            if duration <= 0.1:
                dura_spam += 1
                continue

            # Extract the columns of interest: Date, Time, Caller ID, Cell ID
            date_time_str = parts[0] + ' ' + parts[1]
            caller_id = parts[4]
            callee_id = parts[6]
            cell_id = parts[7]

            # Parse date and time
            date_time = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
            weekday = date_time.weekday()
            hour = date_time.hour

            # Store calls between users
            calls_between_users.append({
                'caller_id': caller_id,
                'callee_id': callee_id,
                'cell_id': cell_id,
                'duration': duration
            })

            # Filter based on time conditions
            if (weekday < 5 and (hour >= 19 or hour <= 6)) or (weekday >= 5):
                if caller_id not in calls_users_time_filter:
                    calls_users_time_filter[caller_id] = {}

                # Count calls per location for significant times
                if cell_id not in calls_users_time_filter[caller_id]:
                    calls_users_time_filter[caller_id][cell_id] = 1
                else:
                    calls_users_time_filter[caller_id][cell_id] += 1
    return calls_users_time_filter, calls_between_users, dura_spam

In [ ]:
def network_radius_calculation_cell_dura(calls_between_users, home_locations, antenna_locations, city, output_suffix, is_log=False, non_zero=False):
    """
    Calculate the network radius based on the calls between users using the caller's cell location
    and the callee's home location, then average.

    Args:
        calls_between_users: List of call records with caller_id, callee_id, and cell_id.
        home_locations: Dictionary mapping users to their home location (cell).
        antenna_locations: Dictionary mapping cell IDs to (latitude, longitude).
        city: The name of the city for saving plots.
        is_log: Whether to apply a log transform to the call weights.
        non_zero: Whether to exclude zero individual radii from the final calculation.

    Returns:
        Average network radius for the city, total weight, number of final users, and number of users with missing home locations.
    """
    # Initialize variables
    individual_radii = []
    total_weight_final = 0.0
    callee_miss_home_record = 0
    zero_radius_count = 0

    # FOR Distribution analysis
    individual_radii_nozero = []

    # Create a dictionary to hold distances and weights per caller
    caller_distances = {}

    # Calculate network radius for each caller
    for call_record in calls_between_users:
        caller_id = call_record['caller_id']
        callee_id = call_record['callee_id']
        caller_cell_id = call_record['cell_id']
        call_weight = call_record['duration']

        # Get caller's location using the cell ID for each call record
        caller_location = antenna_locations.get(caller_cell_id, (np.nan, np.nan))

        # Get callee's home location if available
        callee_homes = home_locations.get(callee_id, None)

        if callee_homes is not None:
            callee_location = antenna_locations.get(callee_homes[0], (np.nan, np.nan))

            # Proceed if both locations are valid
            if not np.isnan(caller_location[0]) and not np.isnan(caller_location[1]) and \
               not np.isnan(callee_location[0]) and not np.isnan(callee_location[1]):

                distance = haversine(*caller_location, *callee_location)

                if distance == 0:
                    # distance = 0.5

                    # FOR Distribution analysis
                    distance_nozero = 0.5
                else:
                    distance_nozero = distance

                # Add distance and weight to caller's record
                if caller_id not in caller_distances:
                    caller_distances[caller_id] = {'distances': [], 'weights': [], 'distances_nozero': []}
                caller_distances[caller_id]['distances'].append(distance * call_weight)
                caller_distances[caller_id]['weights'].append(call_weight)

                # FOR Distribution analysis
                caller_distances[caller_id]['distances_nozero'].append(distance_nozero * call_weight)

        else:
            callee_miss_home_record += 1

    # Calculate the network radius for each caller
    for caller_id, data in caller_distances.items():
        total_weight = sum(data['weights'])
        if total_weight > 0:
            individual_radius = sum(data['distances']) / total_weight

            # FOR Distribution analysis
            individual_radius_nozero = sum(data['distances_nozero']) / total_weight
            individual_radii_nozero.append(individual_radius_nozero)

            if is_log:
                individual_radius = np.log1p(individual_radius)

            if individual_radius == 0:
                zero_radius_count += 1
                if not non_zero:  # Only append if non_zero is False
                    individual_radii.append(individual_radius)
            else:
                individual_radii.append(individual_radius)

            total_weight_final += total_weight

    # Calculate the average network radius
    network_radius = np.mean(individual_radii) if individual_radii else 0
    total_users_final = len(individual_radii)

    # Plot the distribution of individual network radii
    plot_network_radius_distribution(individual_radii, individual_radii_nozero, city, output_suffix)

    return network_radius, total_weight_final, total_users_final, callee_miss_home_record, zero_radius_count

# Method 1 to calculate the radius



1.   Home location for both callers and callees;
2.   Call counts as the weight.



## Distribution analysis for weight(distance here)

In [ ]:
# read cidades_CDR
file_path_cidades = '/content/drive/MyDrive/Brazilian Cities CDR/cidades_CDR.xlsx'
df = pd.read_excel(file_path_cidades)
df['City_lower'] = df['City'].apply(normalize_string)

# loop for loading all the files in the folder
folder_path_cdr = '/content/drive/MyDrive/Brazilian Cities CDR/CDR/'
file_names_cdr = [file for file in os.listdir(folder_path_cdr) if file.endswith('.txt')]

folder_path_ana = '/content/drive/MyDrive/Brazilian Cities CDR/Antena/'
# file_names_ana = [file for file in os.listdir(folder_path_ana) if file.endswith('.txt')]

# take the city name in file names which is between '_' and '.txt'
cities = [file_name.split('_')[1].split('.')[0] for file_name in file_names_cdr]

# Indicator for first time writing csv file
header_written = False

# select the corresponding file in file_names_ana when loop in the file_names_cdr
for city in cities:

    if city == 'Fortaleza':
        continue

    significant_users = set()
    # excluded_users = set()
    cdr_file_name = 'cdr_' + city + '.txt'
    ana_file_name = 'antennas_' + city + '.txt'
    file_path_cdr = os.path.join(folder_path_cdr, cdr_file_name)
    file_path_ana = os.path.join(folder_path_ana, ana_file_name)

    # read antenna locations
    antenna_locations = read_ana(file_path_ana)

    # read cdr
    calls_users_time_filter, calls_between_users = read_cdr_count(file_path_cdr)

    df_calls = convert_dict_to_df_count(calls_users_time_filter)

    # q_low = df_calls['total_call_weight'].quantile(0.1)
    # q_high = df_calls['total_call_weight'].quantile(0.9)
    # q_low = 5
    # q_high = 200

    # print(f'q_low: {q_low}, q_high: {q_high}')

    # df_calls = df_calls[(df_calls['total_call_weight'] >= q_low) & (df_calls['total_call_weight'] <= q_high)]

    # stat, p = normaltest(df_calls['total_call_weight'])
    # normality_result = "Gaussian (fail to reject H0)" if p > 0.05 else "not Gaussian (reject H0)"
    # print(f'Statistics={stat:.3f}, p={p:.3f}')
    # if p > 0.05:
    #     print(f'{city} looks Gaussian (fail to reject H0)')
    # else:
    #     print(f'{city} does not look Gaussian (reject H0)')

    # Plotting the KDE (Kernel Density Estimate)
    plt.figure(figsize=(10, 6))
    if not df_calls['total_call_weight'].empty:
        sns.histplot(df_calls['total_call_weight'], bins=30, kde=False)
        plt.title(f'Frequency Distribution of Call Counts in {city}')
        plt.xlabel('Call Counts')
        plt.ylabel('Frequency')

        # Calculate and display statistical summary
        desc_stats = df_calls['total_call_weight'].describe()
        stats_text = '\n'.join([f'{key}: {value:.2f}' for key, value in desc_stats.items()])
        # stats_text += f'\nNormality Test: {normality_result} (p={p:.3f})'
        plt.figtext(0.35, 0.4, f'Statistical Summary:\n{stats_text}', fontsize=10, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

        # Save the KDE plot
        kde_path = os.path.join('/content/drive/MyDrive/network_radius_city_size/distribution analysis/call counts/all range', f'Call_Counts_{city}.png')
        plt.savefig(kde_path)
        plt.close()
    else:
        print(f"No call weight data available for {city}.")

## Main Loop of Method 1

In [ ]:
# read cidades_CDR
file_path_cidades = '/content/drive/MyDrive/Brazilian Cities CDR/cidades_CDR.xlsx'
df = pd.read_excel(file_path_cidades)
df['City_lower'] = df['City'].apply(normalize_string)

# loop for loading all the files in the folder
folder_path_cdr = '/content/drive/MyDrive/Brazilian Cities CDR/CDR/'
file_names_cdr = [file for file in os.listdir(folder_path_cdr) if file.endswith('.txt')]

folder_path_ana = '/content/drive/MyDrive/Brazilian Cities CDR/Antena/'
# file_names_ana = [file for file in os.listdir(folder_path_ana) if file.endswith('.txt')]

# take the city name in file names which is between '_' and '.txt'
cities = [file_name.split('_')[1].split('.')[0] for file_name in file_names_cdr]

# Indicator for first time writing csv file
header_written = False
is_log = False
non_zero = False
output_suffix = f"call_counts_{'log' if is_log else 'nolog'}_{'nonzero' if non_zero else 'zero'}_home"

# select the corresponding file in file_names_ana when loop in the file_names_cdr
for city in cities:

    if city == 'Fortaleza':
        continue

    significant_users = set()
    # excluded_users = set()
    cdr_file_name = 'cdr_' + city + '.txt'
    ana_file_name = 'antennas_' + city + '.txt'
    file_path_cdr = os.path.join(folder_path_cdr, cdr_file_name)
    file_path_ana = os.path.join(folder_path_ana, ana_file_name)

    # read antenna locations
    antenna_locations = read_ana(file_path_ana)

    # read cdr
    calls_users_time_filter, calls_between_users, dura_spam = read_cdr_count(file_path_cdr)

    # Determine significant users
    significant_users = significant_users_calculation(calls_users_time_filter)

    # Determine the home location based on the filtered records
    home_locations = home_location_calculation(calls_users_time_filter, significant_users)

    count_antenna = len(antenna_locations)
    count_significant_users = len(significant_users)
    filter_users_1 = len(calls_users_time_filter)
    total_users = len(calls_between_users)
    percentage_filter_1 = (filter_users_1 / total_users) * 100 if total_users else 0
    percentage_significant_users = (count_significant_users / total_users) * 100 if total_users else 0
    total_calls_record = sum(sum(calls.values()) for calls in calls_between_users.values()) + dura_spam
    percentage_dura_spam = (dura_spam / total_calls_record) * 100 if total_calls_record else 0


    # release calls_users_time_filter and significant_users
    del calls_users_time_filter
    del significant_users
    gc.collect()

    # ** Calculate the network radius method 1**
    # network_radius, total_weight_final, total_users_final, callee_miss_home = network_radius_calculation(calls_between_users, home_locations, antenna_locations, is_log = is_log)

    # ** Calculation methond 2 **
    network_radius, total_weight_final, total_users_final, callee_miss_home, zero_radius_count = network_radius_calculation_individual(
        calls_between_users, home_locations, antenna_locations, city, output_suffix, is_log=is_log, non_zero=non_zero)

    del calls_between_users
    del home_locations
    del antenna_locations
    gc.collect()

    percentage_zero_radius_count = (zero_radius_count / total_users_final) * 100 if total_users_final else 0
    percentage_final_users = (total_users_final / total_users) * 100 if total_users else 0
    percentage_final_calls = (total_weight_final / total_calls_record) * 100 if total_calls_record else 0

    city_data = (
            network_radius,
            total_users,
            filter_users_1,
            percentage_filter_1,
            count_significant_users,
            percentage_significant_users,
            total_users_final,
            percentage_final_users,
            total_calls_record,
            total_weight_final,
            percentage_final_calls,
            callee_miss_home,
            zero_radius_count,
            percentage_zero_radius_count,
            dura_spam,
            percentage_dura_spam,
            count_antenna
            )

    print(f"City: {city}, Network Radius: {network_radius:.2f} km")

    write_data_to_csv(df, city, city_data, output_suffix)
    header_written = True  # Update the global variable after writing header

# Method 2 to calculate the radius

1.   both home location for callers and callees
2.   Duration as weight


## Distribution analysis for weight(duration here)

In [ ]:
# read cidades_CDR
file_path_cidades = '/content/drive/MyDrive/Brazilian Cities CDR/cidades_CDR.xlsx'
df = pd.read_excel(file_path_cidades)
df['City_lower'] = df['City'].apply(normalize_string)

# loop for loading all the files in the folder
folder_path_cdr = '/content/drive/MyDrive/Brazilian Cities CDR/CDR/'
file_names_cdr = [file for file in os.listdir(folder_path_cdr) if file.endswith('.txt')]

folder_path_ana = '/content/drive/MyDrive/Brazilian Cities CDR/Antena/'
# file_names_ana = [file for file in os.listdir(folder_path_ana) if file.endswith('.txt')]

# take the city name in file names which is between '_' and '.txt'
cities = [file_name.split('_')[1].split('.')[0] for file_name in file_names_cdr]

# Indicator for first time writing csv file
header_written = False

# select the corresponding file in file_names_ana when loop in the file_names_cdr
for city in cities:

    if city == 'Fortaleza':
        continue

    significant_users = set()
    # excluded_users = set()
    cdr_file_name = 'cdr_' + city + '.txt'
    ana_file_name = 'antennas_' + city + '.txt'
    file_path_cdr = os.path.join(folder_path_cdr, cdr_file_name)
    file_path_ana = os.path.join(folder_path_ana, ana_file_name)

    # read antenna locations
    antenna_locations = read_ana(file_path_ana)

    # read cdr
    calls_users_time_filter, calls_users_duration = read_cdr_duration(file_path_cdr)

    df_calls = convert_dict_to_df_count(calls_users_duration)

    # q_low = df_calls['total_call_weight'].quantile(0.1)
    # q_high = df_calls['total_call_weight'].quantile(0.9)
    # q_low = 5
    # q_high = 200

    # print(f'q_low: {q_low}, q_high: {q_high}')

    # df_calls = df_calls[(df_calls['total_call_weight'] >= q_low) & (df_calls['total_call_weight'] <= q_high)]

    stat, p = normaltest(df_calls['total_call_weight'])
    normality_result = "Gaussian (fail to reject H0)" if p > 0.05 else "not Gaussian (reject H0)"
    print(f'Statistics={stat:.3f}, p={p:.3f}')
    if p > 0.05:
        print(f'{city} looks Gaussian (fail to reject H0)')
    else:
        print(f'{city} does not look Gaussian (reject H0)')

    # Plotting the KDE (Kernel Density Estimate)
    plt.figure(figsize=(10, 6))
    if not df_calls['total_call_weight'].empty:
        sns.histplot(df_calls['total_call_weight'], bins=30, kde=False)
        plt.title(f'Frequency Distribution of Call Duration in {city}')
        plt.xlabel('Call duration')
        plt.ylabel('Frequency')

        # Calculate and display statistical summary
        desc_stats = df_calls['total_call_weight'].describe()
        stats_text = '\n'.join([f'{key}: {value:.2f}' for key, value in desc_stats.items()])
        # stats_text += f'\nNormality Test: {normality_result} (p={p:.3f})'
        plt.figtext(0.35, 0.4, f'Statistical Summary:\n{stats_text}', fontsize=10, bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

        # Save the KDE plot
        kde_path = os.path.join('/content/drive/MyDrive/network_radius_city_size/distribution analysis/call duration/all range', f'Call_duration_{city}.png')
        plt.savefig(kde_path)
        plt.close()
    else:
        print(f"No call weight data available for {city}.")

## Main Loop for Method 2

In [ ]:
# read cidades_CDR
file_path_cidades = '/content/drive/MyDrive/Brazilian Cities CDR/cidades_CDR.xlsx'
df = pd.read_excel(file_path_cidades)
df['City_lower'] = df['City'].apply(normalize_string)

# loop for loading all the files in the folder
folder_path_cdr = '/content/drive/MyDrive/Brazilian Cities CDR/CDR/'
file_names_cdr = [file for file in os.listdir(folder_path_cdr) if file.endswith('.txt')]

folder_path_ana = '/content/drive/MyDrive/Brazilian Cities CDR/Antena/'

# take the city name in file names which is between '_' and '.txt'
cities = [file_name.split('_')[1].split('.')[0] for file_name in file_names_cdr]

# Indicator for first time writing csv file
header_written = False
is_log = False
non_zero = False
output_suffix = f"dura_{'log' if is_log else 'nolog'}_{'nonzero' if non_zero else 'zero'}_home"

# select the corresponding file in file_names_ana when loop in the file_names_cdr
for city in cities:

    if city == 'Fortaleza':
        continue

    significant_users = set()
    cdr_file_name = 'cdr_' + city + '.txt'
    ana_file_name = 'antennas_' + city + '.txt'
    file_path_cdr = os.path.join(folder_path_cdr, cdr_file_name)
    file_path_ana = os.path.join(folder_path_ana, ana_file_name)

    # Check if cdr and ana files are in the folders
    if cdr_file_name not in os.listdir(folder_path_cdr):
        print(f"File {cdr_file_name} not found in the folder.")
        continue

    if ana_file_name not in os.listdir(folder_path_ana):
        print(f"File {ana_file_name} not found in the folder.")
        continue

    # read antenna locations
    antenna_locations = read_ana(file_path_ana)

    # read cdr
    calls_users_time_filter, calls_users_duration, dura_spam = read_cdr_duration(file_path_cdr)

    # Determine significant users
    significant_users = significant_users_calculation(calls_users_time_filter)

    # Determine the home location based on the filtered records
    home_locations = home_location_calculation(calls_users_time_filter, significant_users)

    count_antenna = len(antenna_locations)
    total_users = len(calls_users_duration)
    total_calls_weight = sum(sum(calls.values()) for calls in calls_users_duration.values())

    filter_users_1 = len(calls_users_time_filter)
    percentage_filter_1 = (filter_users_1 / total_users) * 100 if total_users else 0

    count_significant_users = len(significant_users)
    percentage_significant_users = (count_significant_users / total_users) * 100 if total_users else 0

    # release calls_users_time_filter and significant_users
    del calls_users_time_filter
    del significant_users
    gc.collect()

    # Calculate the network radius
    network_radius, total_weight_final, total_users_final, callee_miss_home, zero_radius_count = network_radius_calculation_individual(
        calls_users_duration, home_locations, antenna_locations, city, output_suffix, is_log=is_log, non_zero=non_zero)

    del calls_users_duration
    del home_locations
    del antenna_locations
    gc.collect()

    percentage_zero_radius_count = (zero_radius_count / total_users_final) * 100 if total_users_final else 0
    percentage_final_users = (total_users_final / total_users) * 100 if total_users else 0
    percentage_final_calls = (total_weight_final / total_calls_weight) * 100 if total_calls_weight else 0

    city_data = (
            network_radius,
            total_users,
            filter_users_1,
            percentage_filter_1,
            count_significant_users,
            percentage_significant_users,
            total_users_final,
            percentage_final_users,
            total_calls_weight,
            total_weight_final,
            percentage_final_calls,
            callee_miss_home,
            zero_radius_count,
            percentage_zero_radius_count,
            dura_spam,
            0,
            count_antenna
            )

    print(f"City: {city}, Network Radius: {network_radius:.2f} km")

    write_data_to_csv(df, city, city_data, output_suffix)
    header_written = True  # Update the global variable after writing header

# Method 3 to calculate the radius

1.   Cell location for callers and home location for callees
2.   Call counts as weight

In [ ]:
# read cidades_CDR
file_path_cidades = '/content/drive/MyDrive/Brazilian Cities CDR/cidades_CDR.xlsx'
df = pd.read_excel(file_path_cidades)
df['City_lower'] = df['City'].apply(normalize_string)

# loop for loading all the files in the folder
folder_path_cdr = '/content/drive/MyDrive/Brazilian Cities CDR/CDR/'
file_names_cdr = [file for file in os.listdir(folder_path_cdr) if file.endswith('.txt')]

folder_path_ana = '/content/drive/MyDrive/Brazilian Cities CDR/Antena/'
# file_names_ana = [file for file in os.listdir(folder_path_ana) if file.endswith('.txt')]

# take the city name in file names which is between '_' and '.txt'
cities = [file_name.split('_')[1].split('.')[0] for file_name in file_names_cdr]

# Indicator for first time writing csv file
header_written = False

is_log = False
non_zero = False
output_suffix = f"call_counts_{'log' if is_log else 'nolog'}_{'nonzero' if non_zero else 'zero'}_cell"

# select the corresponding file in file_names_ana when loop in the file_names_cdr
for city in cities:

    if city == 'Fortaleza':
        continue

    significant_users = set()
    cdr_file_name = 'cdr_' + city + '.txt'
    ana_file_name = 'antennas_' + city + '.txt'
    file_path_cdr = os.path.join(folder_path_cdr, cdr_file_name)
    file_path_ana = os.path.join(folder_path_ana, ana_file_name)

    # read antenna locations
    antenna_locations = read_ana(file_path_ana)

    # read cdr
    calls_users_time_filter, calls_between_users, dura_spam = read_cdr_count_cell(file_path_cdr)

    # Determine significant users
    significant_users = significant_users_calculation(calls_users_time_filter)

    # Determine the home location based on the filtered records
    home_locations = home_location_calculation(calls_users_time_filter, significant_users)

    count_antenna = len(antenna_locations)
    count_significant_users = len(significant_users)
    filter_users_1 = len(calls_users_time_filter)
    total_users = len(set(record['caller_id'] for record in calls_between_users))
    percentage_filter_1 = (filter_users_1 / total_users) * 100 if total_users else 0
    percentage_significant_users = (count_significant_users / total_users) * 100 if total_users else 0
    total_calls_record = len(calls_between_users) + dura_spam
    percentage_dura_spam = (dura_spam / total_calls_record) * 100 if total_calls_record else 0

    # release calls_users_time_filter and significant_users
    del calls_users_time_filter
    del significant_users
    gc.collect()

    network_radius, total_weight_final, total_users_final, callee_miss_home, zero_radius_count = network_radius_calculation_cell(
        calls_between_users=calls_between_users, home_locations=home_locations, antenna_locations=antenna_locations,
        city=city, output_suffix=output_suffix, is_log=is_log, non_zero=non_zero)

    del calls_between_users
    del home_locations
    del antenna_locations
    gc.collect()

    percentage_zero_radius_count = (zero_radius_count / total_users_final) * 100 if total_users_final else 0
    percentage_final_users = (total_users_final / total_users) * 100 if total_users else 0
    percentage_final_calls = (total_weight_final / total_calls_record) * 100 if total_calls_record else 0

    city_data = (
            network_radius,
            total_users,
            filter_users_1,
            percentage_filter_1,
            count_significant_users,
            percentage_significant_users,
            total_users_final,
            percentage_final_users,
            total_calls_record,
            total_weight_final,
            percentage_final_calls,
            callee_miss_home,
            zero_radius_count,
            percentage_zero_radius_count,
            dura_spam,
            percentage_dura_spam,
            count_antenna
            )

    print(f"City: {city}, Network Radius: {network_radius:.2f} km")

    write_data_to_csv(df, city, city_data, output_suffix)
    header_written = True  # Update the global variable after writing header

# Method 4 to calculate the radius

1.   Cell location for callers and home location for callees
2.   Call duration as weight

In [ ]:
# read cidades_CDR
file_path_cidades = '/content/drive/MyDrive/Brazilian Cities CDR/cidades_CDR.xlsx'
df = pd.read_excel(file_path_cidades)
df['City_lower'] = df['City'].apply(normalize_string)

# loop for loading all the files in the folder
folder_path_cdr = '/content/drive/MyDrive/Brazilian Cities CDR/CDR/'
file_names_cdr = [file for file in os.listdir(folder_path_cdr) if file.endswith('.txt')]

folder_path_ana = '/content/drive/MyDrive/Brazilian Cities CDR/Antena/'
# file_names_ana = [file for file in os.listdir(folder_path_ana) if file.endswith('.txt')]

# take the city name in file names which is between '_' and '.txt'
cities = [file_name.split('_')[1].split('.')[0] for file_name in file_names_cdr]

# Indicator for first time writing csv file
header_written = False
is_log = False
non_zero = False
output_suffix = f"dura_{'log' if is_log else 'nolog'}_{'nonzero' if non_zero else 'zero'}_cell"

# select the corresponding file in file_names_ana when loop in the file_names_cdr
for city in cities:

    if city == 'Fortaleza':
        continue

    significant_users = set()
    cdr_file_name = 'cdr_' + city + '.txt'
    ana_file_name = 'antennas_' + city + '.txt'
    file_path_cdr = os.path.join(folder_path_cdr, cdr_file_name)
    file_path_ana = os.path.join(folder_path_ana, ana_file_name)

    # read antenna locations
    antenna_locations = read_ana(file_path_ana)

    # read cdr
    calls_users_time_filter, calls_between_users, dura_spam = read_cdr_count_cell_dura(file_path_cdr)

    # Determine significant users
    significant_users = significant_users_calculation(calls_users_time_filter)

    # Determine the home location based on the filtered records
    home_locations = home_location_calculation(calls_users_time_filter, significant_users)

    count_antenna = len(antenna_locations)
    count_significant_users = len(significant_users)
    filter_users_1 = len(calls_users_time_filter)
    total_users = len(set(record['caller_id'] for record in calls_between_users))
    percentage_filter_1 = (filter_users_1 / total_users) * 100 if total_users else 0
    percentage_significant_users = (count_significant_users / total_users) * 100 if total_users else 0
    total_calls_record = len(calls_between_users) + dura_spam
    percentage_dura_spam = (dura_spam / total_calls_record) * 100 if total_calls_record else 0

    # release calls_users_time_filter and significant_users
    del calls_users_time_filter
    del significant_users
    gc.collect()

    network_radius, total_weight_final, total_users_final, callee_miss_home, zero_radius_count = network_radius_calculation_cell_dura(
        calls_between_users=calls_between_users, home_locations=home_locations, antenna_locations=antenna_locations,
        city=city, output_suffix=output_suffix, is_log=is_log, non_zero=non_zero)

    # Calculate the network radius
    # network_radius, total_weight_final, total_users_final, callee_miss_home = network_radius_calculation_cell(
        # calls_between_users, home_locations, antenna_locations, is_log = is_log)

    del calls_between_users
    del home_locations
    del antenna_locations
    gc.collect()

    percentage_zero_radius_count = (zero_radius_count / total_users_final) * 100 if total_users_final else 0
    percentage_final_users = (total_users_final / total_users) * 100 if total_users else 0
    percentage_final_calls = (total_weight_final / total_calls_record) * 100 if total_calls_record else 0

    city_data = (
            network_radius,
            total_users,
            filter_users_1,
            percentage_filter_1,
            count_significant_users,
            percentage_significant_users,
            total_users_final,
            percentage_final_users,
            total_calls_record,
            total_weight_final,
            percentage_final_calls,
            callee_miss_home,
            zero_radius_count,
            percentage_zero_radius_count,
            dura_spam,
            percentage_dura_spam,
            count_antenna
            )

    print(f"City: {city}, Network Radius: {network_radius:.2f} km")

    write_data_to_csv(df, city, city_data, output_suffix)
    header_written = True  # Update the global variable after writing header

# Dataframe Method

## Common Utility Functions

In [ ]:
# Read CDR data into a DataFrame
def load_cdr_data(file_path_cdr):
    """

    Args:
      file_path_cdr:

    Returns:
      Dataframe of CDR data

    """
    columns_to_use = [0, 1, 2, 4, 6, 7]
    column_names = ['date', 'time', 'duration', 'caller_id', 'callee_id', 'cell_id']

    # Specify data types for each column to optimize memory usage
    dtype = {
        'duration': 'float32',
        'caller_id': 'category',
        'callee_id': 'category',
        'cell_id': 'category'
    }

    # Read the data including only the required columns and specifying the date-time format
    df_cdr = pd.read_csv(
        file_path_cdr,
        sep=';',
        header=None,
        usecols=columns_to_use,
        dtype=dtype,
        names=column_names,
        parse_dates={'datetime': ['date', 'time']},
        date_format='%Y-%m-%d %H:%M:%S'
    )

    # Add a 'weekday' and 'hour' column for time filtering
    df_cdr['weekday'] = df_cdr['datetime'].dt.weekday
    df_cdr['hour'] = df_cdr['datetime'].dt.hour

    return df_cdr

In [ ]:
# Filter for significant time conditions and aggregate call data
def filter_significant_calls(df_cdr):
    """

    Args:
      df_cdr:

    Returns:
      List of significant users and dictionary of their home locations

    """
    # Filter based on time conditions (weekdays after 7 PM and before 6 AM, weekends all day)
    significant_calls = df_cdr[((df_cdr['weekday'] < 5) & ((df_cdr['hour'] >= 19) | (df_cdr['hour'] <= 6))) | (df_cdr['weekday'] >= 5)]

    filter_users_1 = significant_calls['caller_id'].nunique()

    # Group by caller and cell_id to get call counts
    calls_users_time_filter = significant_calls.groupby(['caller_id', 'cell_id']).size().reset_index(name='call_count')

    # Identify significant users
    significant_users = calls_users_time_filter.groupby('caller_id')['call_count'].sum()
    significant_users = significant_users[(significant_users >= 5) & (significant_users <= 200)].index.tolist()

    # Determine home locations based on most frequent cell_id for each significant user
    home_locations = calls_users_time_filter[calls_users_time_filter['caller_id'].isin(significant_users)]

    home_locations = home_locations.loc[home_locations.groupby('caller_id', observed=True)['call_count'].idxmax()]

    # home_locations = home_locations.loc[home_locations.groupby('caller_id')['call_count'].idxmax()]

    return home_locations.set_index('caller_id')['cell_id'].to_dict(), significant_users, filter_users_1

In [ ]:
# # Calculate the network radius
# def calculate_network_radius(df_cdr, home_locations, antenna_locations, weight='call_count', is_log=False, location='home'):
#     """
#     Calculate the network radius based on calls between users and their home locations

#     Args:
#       df_cdr: Dataframe of CDR data
#       home_locations: Dictionary of significant users and their home locations
#       antenna_locations: Dictionary of cells' locations
#       weight: 'call_count' or 'duration' to use different value as weight (default 'counts')
#       is_log: 'Ture' or 'False' to use log1p transform or not (default 'False')
#       location: 'home' or 'cell' to use home locations or actual location of each call record for callers,
#             always use home location for callees (default 'home')

#     Returns:
#       Network radius, total weight, total users, and number of users with missing home locations
#     """
#     total_weight_final = 0.0
#     callee_miss_home = 0

#     # Simplify dataframe by remove redundant columns
#     if location == 'home':
#         if weight == 'call_count':
#             df_cdr = df_cdr[['caller_id', 'callee_id']]
#         else:
#             df_cdr = df_cdr[['caller_id', 'callee_id', 'duration']]
#     else:
#         if weight == 'call_count':
#             df_cdr = df_cdr[['caller_id', 'callee_id', 'cell_id']]
#         else:
#             df_cdr = df_cdr[['caller_id', 'callee_id', 'cell_id', 'duration']]

#     # Use vectorized operations and avoid apply when possible
#     def get_location(row, type_):
#         if type_ == 'caller':
#             return home_locations.get(row['caller_id'], (np.nan, np.nan)) if location == 'home' else antenna_locations.get(row['cell_id'], (np.nan, np.nan))
#         else:
#             return home_locations.get(row['callee_id'], (np.nan, np.nan))

#     df_cdr[['caller_lat', 'caller_lon']] = df_cdr.apply(lambda row: pd.Series(get_location(row, 'caller')), axis=1)
#     df_cdr[['callee_lat', 'callee_lon']] = df_cdr.apply(lambda row: pd.Series(get_location(row, 'callee')), axis=1)

#     df_cdr['distance'] = df_cdr.apply(lambda row: haversine(row['caller_lat'], row['caller_lon'], row['callee_lat'], row['callee_lon']), axis=1)

#     # Count the number of unique callees with missing home locations
#     callee_miss_home = len(pd.Index(df_cdr['callee_id'].unique()).difference(home_locations.keys()))

#     # Filter for rows where both caller and callee have locations
#     valid_calls = df_cdr.dropna(subset=['distance'])

#     # Compute call_count if required
#     if weight == 'call_count':
#         # Count occurrences of each (caller_id, callee_id) pair in valid_calls
#         call_counts = valid_calls.groupby(['caller_id', 'callee_id']).size().reset_index(name='call_count')
#         valid_calls = valid_calls.merge(call_counts, on=['caller_id', 'callee_id'], how='left')

#     # Choose the weight column
#     weight_column = 'duration' if weight == 'duration' else 'call_count'

#     # Apply log transform to the weight if specified
#     if is_log:
#         valid_calls[weight_column] = np.log1p(valid_calls[weight_column])

#     valid_calls['weighted_distance'] = valid_calls['distance'] * valid_calls[weight_column]
#     total_distance_weighted = valid_calls['weighted_distance'].sum()
#     total_weight_final = valid_calls[weight_column].sum()

#     network_radius = total_distance_weighted / valid_calls['caller_id'].nunique() if valid_calls['caller_id'].nunique() else 0

#     return network_radius, total_weight_final, valid_calls['caller_id'].nunique(), callee_miss_home

In [ ]:
# Calculate the network radius
def calculate_network_radius(df_cdr, home_locations, antenna_locations, weight='call_count', is_log=False, location='home'):
    """
    Calculate the network radius based on calls between users and their home locations.

    Args:
      df_cdr: Dataframe of CDR data
      home_locations: Dictionary of significant users and their home locations
      antenna_locations: Dictionary of cells' locations
      weight: 'call_count' or 'duration' to use different value as weight (default 'counts')
      is_log: 'True' or 'False' to use log1p transform or not (default 'False')
      location: 'home' or 'cell' to use home locations or actual location of each call record for callers,
                always use home location for callees (default 'home')

    Returns:
      Network radius, total weight, total users, and number of users with missing home locations
    """
    total_weight_final = 0.0

    # Simplify dataframe by removing redundant columns
    if location == 'home':
        if weight == 'call_count':
            # Group by caller and callee to get call counts, and reduce to unique pairs
            call_counts = df_cdr.groupby(['caller_id', 'callee_id']).size().reset_index(name='call_count')
            df_cdr = call_counts
        else:
            df_cdr = df_cdr[['caller_id', 'callee_id', 'duration']]
    else:
        if weight == 'call_count':
            call_counts = df_cdr.groupby(['caller_id', 'callee_id', 'cell_id']).size().reset_index(name='call_count')
            df_cdr = call_counts
        else:
            df_cdr = df_cdr[['caller_id', 'callee_id', 'cell_id', 'duration']]

    # Add locations based on home or cell
    def get_location(type_, row):
        if type_ == 'caller':
            return home_locations.get(row['caller_id']) if location == 'home' else antenna_locations.get(row['cell_id'])
        else:
            return home_locations.get(row['callee_id'])

    # Calculate distances and filter valid calls
    df_cdr['caller_location'] = df_cdr.apply(lambda row: get_location('caller', row), axis=1)
    df_cdr['callee_location'] = df_cdr.apply(lambda row: get_location('callee', row), axis=1)

    # Drop rows with missing locations
    valid_calls = df_cdr.dropna(subset=['caller_location', 'callee_location'])

    # Calculate distance using vectorized operations
    valid_calls['distance'] = valid_calls.apply(
        lambda row: haversine(*(row['caller_location']), *(row['callee_location'])),
        axis=1
    )

    # Select weight column
    weight_column = 'duration' if weight == 'duration' else 'call_count'

    # Apply log transform if specified
    if is_log:
        valid_calls[weight_column] = np.log1p(valid_calls[weight_column])

    # Calculate weighted distances
    valid_calls['weighted_distance'] = valid_calls['distance'] * valid_calls[weight_column]
    total_distance_weighted = valid_calls['weighted_distance'].sum()
    total_weight_final = valid_calls[weight_column].sum()

    # Calculate network radius
    network_radius = total_distance_weighted / valid_calls['caller_id'].nunique() if valid_calls['caller_id'].nunique() else 0

    # Count missing home locations for callees
    unique_callees = pd.Series(df_cdr['callee_id'].unique())
    callee_miss_home = len(unique_callees) - unique_callees.isin(home_locations.keys()).sum()

    return network_radius, total_weight_final, valid_calls['caller_id'].nunique(), callee_miss_home

In [ ]:
# Main loop to process each city
def process_cities(weight, is_log, location, output_suffix):
    """
    Main loop to process each city with given parameters for network radius calculation.

    Args:
      weight: 'call_count' or 'duration'
      is_log: True or False
      location: 'home' or 'cell'
      output_suffix: Suffix for the output file name to differentiate parameter sets
    """
    # Read city data
    file_path_cidades = '/content/drive/MyDrive/Brazilian Cities CDR/cidades_CDR.xlsx'
    df_city = pd.read_excel(file_path_cidades)
    df_city['City_lower'] = df_city['City'].apply(normalize_string)

    folder_path_cdr = '/content/drive/MyDrive/Brazilian Cities CDR/CDR/'
    folder_path_ana = '/content/drive/MyDrive/Brazilian Cities CDR/Antena/'
    file_names_cdr = [file for file in os.listdir(folder_path_cdr) if file.endswith('.txt')]
    cities = [file_name.split('_')[1].split('.')[0] for file_name in file_names_cdr]

    header_written = False

    for city in cities:
        if city == 'Fortaleza':
            continue

        # Load data for the city
        file_path_cdr = os.path.join(folder_path_cdr, f'cdr_{city}.txt')
        file_path_ana = os.path.join(folder_path_ana, f'antennas_{city}.txt')
        df_cdr = load_cdr_data(file_path_cdr)
        antenna_locations = read_ana(file_path_ana)

        # Process data
        home_locations, significant_users, filter_users_1 = filter_significant_calls(df_cdr)
        network_radius, total_weight_final, total_users_final, callee_miss_home = calculate_network_radius(
            df_cdr, home_locations, antenna_locations, weight=weight, is_log=is_log, location=location
        )

        # Calculate percentages and other metrics
        total_users = df_cdr['caller_id'].nunique()
        count_significant_users = len(significant_users)
        percentage_filter_1 = (filter_users_1 / total_users) * 100 if total_users else 0
        percentage_significant_users = (count_significant_users / total_users) * 100 if total_users else 0
        total_calls_record = df_cdr.shape[0]
        percentage_final_users = (total_users_final / total_users) * 100 if total_users else 0
        percentage_final_calls = (total_weight_final / total_calls_record) * 100 if total_calls_record else 0

        del df_cdr, antenna_locations
        gc.collect()

        city_data = (
            network_radius,
            total_users,
            filter_users_1,
            percentage_filter_1,
            count_significant_users,
            percentage_significant_users,
            total_users_final,
            percentage_final_users,
            total_calls_record,
            total_weight_final,
            percentage_final_calls,
            callee_miss_home
        )

        print(f"City: {city}, Network Radius: {network_radius:.2f} km")

        # Write data to CSV
        write_data_to_csv(df_city, city, city_data, output_suffix, header_written)
        header_written = True

In [ ]:
# Run the process for all combinations of parameters
weights = ['call_count', 'duration']
logs = [False, True]
locations = ['home', 'cell']

# List to store output suffixes
output_suffixes = []

for weight in weights:
    for is_log in logs:
        for location in locations:
            output_suffix = f"{weight}_{'log' if is_log else 'nolog'}_{location}"
            output_suffixes.append(output_suffix)
            process_cities(weight, is_log, location, output_suffix)

# Result Analysis

In [ ]:
# for output_suffix in output_suffixes:
#     # Define the main directory and subdirectory paths
#     main_directory = '/content/drive/MyDrive/network_radius_city_size'
#     subdirectory = os.path.join(main_directory, output_suffix)

#     # Create the subdirectory if it does not exist
#     os.makedirs(subdirectory, exist_ok=True)

#     # Construct the full file path
#     file_path = os.path.join(subdirectory, f'network_radius_{output_suffix}.csv')

#     data = pd.read_csv(file_path)


#     # Select numeric columns
#     numeric_data = data.select_dtypes(include=[float, int])

#     cols_to_select = list(range(0, 1)) + list(range(12, 21)) # not include left range
#     filter_data = numeric_data.iloc[:, cols_to_select]

#     # Calculate the correlation matrix
#     correlation_matrix = filter_data.corr()

#     plt.figure(figsize=(10, 8))
#     sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
#     plt.title('Correlation Matrix')
#     plt.savefig(os.path.join(subdirectory, f'correlation_matrix_{output_suffix}.png'))
#     plt.show()

#     # Scatterplot: City area and Social Network radius
#     plt.figure(figsize=(10, 6))
#     sns.scatterplot(x='surfaceOfAdministrativeArea km2', y='Network Radius', data=data)

#     # Label city names
#     for i in range(data.shape[0]):
#         plt.text(x=data['surfaceOfAdministrativeArea km2'][i],
#                 y=data['Network Radius'][i],
#                 s=data['City'][i],
#                 fontdict=dict(color='red',size=10)
#                 )

#     plt.title('City Area vs. Network Radius')
#     plt.xlabel('Surface of Administrative Area (km2)')
#     plt.ylabel('Network Radius (km)')
#     plt.savefig(os.path.join(subdirectory, f'city_area_vs_network_radius_{output_suffix}.png'))
#     plt.show()

#     # Scatterplot: Population and Social Network radius
#     plt.figure(figsize=(10, 6))
#     sns.scatterplot(x='Population 2010', y='Network Radius', data=data)

#     # Label city names
#     for i in range(data.shape[0]):
#         plt.text(x=data['Population 2010'][i],
#                 y=data['Network Radius'][i],
#                 s=data['City'][i],
#                 fontdict=dict(color='red',size=10))

#     plt.title('Population 2010 vs. Network Radius')
#     plt.xlabel('Population 2010 (HAB)')
#     plt.ylabel('Network Radius (km)')
#     plt.savefig(os.path.join(subdirectory, f'population_2010_vs_network_radius_{output_suffix}.png'))
#     plt.show()

#     # Regression plot: city size vs. social network radius
#     plt.figure(figsize=(10, 6))
#     sns.regplot(x='surfaceOfAdministrativeArea km2', y='Network Radius', data=data)
#     plt.title('City Area vs. Network Radius with Regression Line')
#     plt.xlabel('Surface of Administrative Area (km2)')
#     plt.ylabel('Network Radius (km)')
#     plt.savefig(os.path.join(subdirectory, f'city_area_vs_network_radius_regression_{output_suffix}.png'))
#     plt.show()

#     # Regression plot: population vs. social network radius
#     plt.figure(figsize=(10, 6))
#     sns.regplot(x='Population 2010', y='Network Radius', data=data)
#     plt.title('Population 2010 vs. Network Radius with Regression Line')
#     plt.xlabel('Population 2010 (HAB)')
#     plt.ylabel('Network Radius (km)')
#     plt.savefig(os.path.join(subdirectory, f'population_2010_vs_network_radius_regression_{output_suffix}.png'))
#     plt.show()


In [ ]:
# Define the main directory and subdirectory paths
main_directory = '/content/drive/MyDrive/network_radius_city_size'
subdirectory = os.path.join(main_directory, output_suffix)

# Create the subdirectory if it does not exist
os.makedirs(subdirectory, exist_ok=True)

# Construct the full file path
file_path = os.path.join(subdirectory, f'network_radius_{output_suffix}.csv')

print(file_path)

data = pd.read_csv(file_path)

In [ ]:
numeric_data = data.select_dtypes(include=[float, int])


cols_to_select = list(range(0, 1)) + list(range(17, 26)) # not include left range
filter_data = numeric_data.iloc[:, cols_to_select]

correlation_matrix = filter_data.corr()

print(correlation_matrix)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.savefig(os.path.join(subdirectory, f'correlation_matrix_{output_suffix}.png'))
plt.show()

In [ ]:
# Regression plot: city size vs. social network radius
plt.figure(figsize=(10, 6))
sns.regplot(x='surfaceOfAdministrativeArea km2', y='Network Radius', data=data)
plt.title('City Area vs. Network Radius with Regression Line')
plt.xlabel('Surface of Administrative Area (km2)')
plt.ylabel('Network Radius (km)')
plt.savefig(os.path.join(subdirectory, f'area_regression_{output_suffix}.png'))
plt.show()

# Regression plot: population vs. social network radius
plt.figure(figsize=(10, 6))
sns.regplot(x='Population 2010', y='Network Radius', data=data)
plt.title('Population 2010 vs. Network Radius with Regression Line')
plt.xlabel('Population 2010 (HAB)')
plt.ylabel('Network Radius (km)')
plt.savefig(os.path.join(subdirectory, f'population_regression_{output_suffix}.png'))
plt.show()

In [ ]:
# Scatterplot: City area and Social Network radius
plt.figure(figsize=(10, 6))
sns.scatterplot(x='surfaceOfAdministrativeArea km2', y='Network Radius', data=data)

# Label city names
for i in range(data.shape[0]):
    plt.text(x=data['surfaceOfAdministrativeArea km2'][i],
             y=data['Network Radius'][i],
             s=data['City'][i],
             fontdict=dict(color='red',size=10)
             )

plt.title('City Area vs. Network Radius')
plt.xlabel('Surface of Administrative Area (km2)')
plt.ylabel('Network Radius (km)')
plt.savefig(os.path.join(subdirectory, f'area_scatter_{output_suffix}.png'))
plt.show()

# Scatterplot: Population and Social Network radius
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Population 2010', y='Network Radius', data=data)

# Label city names
for i in range(data.shape[0]):
    plt.text(x=data['Population 2010'][i],
             y=data['Network Radius'][i],
             s=data['City'][i],
             fontdict=dict(color='red',size=10))

plt.title('Population 2010 vs. Network Radius')
plt.xlabel('Population 2010 (HAB)')
plt.ylabel('Network Radius (km)')
plt.savefig(os.path.join(subdirectory, f'population_scatter_{output_suffix}.png'))
plt.show()

In [ ]:
import statsmodels.api as sm

# Selection of independent and dependent variables
X = numeric_data[['Population 2010', 'surfaceOfAdministrativeArea km2']]
y = numeric_data['Network Radius']

# Adding a constant term
X = sm.add_constant(X)

# fit a regression model
model = sm.OLS(y, X).fit()

# Output regression results
print(model.summary())

with open(os.path.join(subdirectory, f'regression_summary_{output_suffix}.txt'), "w") as f:
    f.write(model.summary().as_text())
    f.write("\n")
    f.write(model.summary().as_latex())

# Plot the heatmap and antenna locations

In [ ]:
import folium
from folium.plugins import HeatMap, MarkerCluster

locations = []
for user, cellid in home_locations.items():
    if cellid in antenna_locations:
        lat, lon = antenna_locations[cellid]
        locations.append([lat, lon])

map_center = [locations[0][0], locations[0][1]]
map = folium.Map(location=map_center, zoom_start=12)

HeatMap(locations).add_to(map)

map.save('home_location_heatmap.html')

In [ ]:
import folium
from folium.plugins import HeatMap, MarkerCluster

map_center = list(antenna_locations.values())[0]

m = folium.Map(location=map_center, zoom_start=10)

marker_cluster = MarkerCluster().add_to(m)

for cellid, (lat, lon) in antenna_locations.items():
    folium.Marker(
        location=[lat, lon],
        popup=str(cellid),
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)

m.save('antennas_locations_map_with_clusters.html')

In [ ]:
import folium
from folium.plugins import HeatMap, MarkerCluster

# Collect latitudes and longitudes for all home locations for the heatmap
locations = []

# Calculate the number of occurrences of each cellid to be used as the number of users
cellid_counts = {}

for user, cellid in home_locations.items():
    if cellid in antenna_locations:
        lat, lon = antenna_locations[cellid]
        locations.append([lat, lon])
        cellid_counts[cellid] = cellid_counts.get(cellid, 0) + 1

# If there are home locations, choose the first one as the map center; otherwise, choose the first antenna locations
map_center = locations[0] if locations else list(antenna_locations.values())[0]

# Create a folium map instance
m = folium.Map(location=map_center, zoom_start=12)

# Add a heatmap layer for home locations
HeatMap(locations).add_to(m)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(m)

# Add markers for all antenna locations
for cellid, (lat, lon) in antenna_locations.items():
    # Get the users number using cellid_counts
    users_count = cellid_counts.get(cellid, 0)

    popup_text = f"Cell ID: {cellid}<br>Users: {users_count}"
    folium.Marker(
        location=[lat, lon],
        popup=popup_text,  # Popup shows the popup_text
        icon=folium.Icon(color='blue', icon='info-sign')  # Customize marker style
    ).add_to(marker_cluster)

# Save the map as an HTML file
m.save('map_with_heatmap_and_clusters.html')

# Calculate home location, social size and plot with filter

### Extract call records without filter and with filter

### Caculate the social range for each user

In [ ]:
# TODO: Do we need to filter the call made from outside the home location?

# Social range
social_ranges = {}

# Dictionary to hold the distance between
network_edges = {}

for line in open(file_path_cdr_caucaia, 'r'):
    parts = line.strip().split(';')
    caller_id = parts[4]
    called_id = parts[6]

    if caller_id in home_locations and called_id in home_locations:
        caller_home = home_locations[caller_id]
        called_home = home_locations[called_id]
        if caller_home in antenna_locations and called_home in antenna_locations:
            lat1, lon1 = antenna_locations[caller_home]
            lat2, lon2 = antenna_locations[called_home]
            distance = haversine(lat1, lon1, lat2, lon2)

            if caller_id not in network_edges:
                network_edges[caller_id] = {}
            if called_id not in network_edges[caller_id]:
                network_edges[caller_id][called_id] = distance
            else:
                network_edges[caller_id][called_id] += distance

# Sum distances and calculate average range
for caller, calls in network_edges.items():
    total_distance = sum(calls.values())
    total_calls = sum(user_calls[caller].values())
    social_ranges[caller] = total_distance / total_calls if total_calls else 0

# Output results
for user, range_km in social_ranges.items():
    print(f'User {user} has a social range of {range_km:.2f} km')

## 1.1 Extract each user's call records {(User ID -> {CELLID -> Number of Calls}}

#### 1.3 Plot the distribution and boxplot of the total number of phone call

In [ ]:
# Plot the distribution
total_calls_users = [sum(calls.values()) for calls in user_calls.values()]

plt.figure(figsize=(10, 6))
plt.hist(total_calls_users, bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of Total Call Counts per User')
plt.xlabel('Total Call Counts')
plt.ylabel('Number of Users')
plt.grid(axis='y', alpha=0.75)
# plt.xscale('log')

plt.show()

In [ ]:
plt.boxplot(total_calls_users, vert=True, patch_artist=True,
            flierprops=dict(marker='o', markerfacecolor='red', markersize=3),
            showmeans=True)

plt.title('Boxplot of Total Call Counts per User')
plt.ylabel('Total Call Counts')
# plt.yscale('log')

In [ ]:
pd.DataFrame(total_calls_users).describe()

In [ ]:
# Calculate the number of unique callees per caller
unique_calls_per_user = {user: len(calls) for user, calls in social_network.items()}

# Count how many users called exactly 1, 2, 3, 4, 5 unique users
call_count_distribution = collections.Counter(unique_calls_per_user.values())

print(f"We have totally {len(unique_calls_per_user)} users who made call.")

total_1_to_5 = sum(call_count_distribution[i] for i in range(1, 6))
print(f"Number of users calling between 1 and 5 unique people: {total_1_to_5}")

percentage_1_to_5 = (total_1_to_5 / total_users) * 100
print(f"Percentage of users calling between 1 and 5 unique people: {percentage_1_to_5:.2f}%")

for i in range(1, 6):
    print(f"Number of unique users call {i} unique people: {call_count_distribution[i]}")

# Dataframe analysis for smaller cities

In [ ]:
# read CDR data to daraframe
file_path_cdr = '/content/drive/MyDrive/Brazilian Cities CDR/CDR/cdr_Caucaia.txt'

# Define the indices and names of the columns to be used
columns_to_use = [0, 1, 2, 4, 6, 7]  # Adjust according to your actual data structure
column_names = ['date', 'time', 'duration', 'caller_id', 'callee_id', 'cell_id']

# Specify data types for each column to optimize memory usage
dtype = {
    'duration': 'float32',  # Assuming duration is a floating number
    'caller_id': 'category',  # Use 'category' data type to reduce memory usage for repetitive strings
    'callee_id': 'category',
    'cell_id': 'category'
}

# Read the data including only the required columns and specifying the date-time format
df_cdr = pd.read_csv(
    file_path_cdr,
    sep=';',
    header=None,
    usecols=columns_to_use,
    dtype=dtype,
    names=column_names,
    parse_dates=[['date', 'time']],
    date_format='%Y-%m-%d %H:%M:%S'
)

print(df_cdr.head())

# read antenna locations
# file_path_ana = '/content/drive/MyDrive/Brazilian Cities CDR/Antena/antennas_Franca.txt'
# df_ana = read_ana(file_path_ana)



In [ ]:
df_cdr.info()

In [ ]:
start_date = df_cdr['date_time'].min()
end_date = df_cdr['date_time'].max()
total_days = (end_date - start_date).days + 1

print(f"Date range: {start_date} to {end_date}, {total_days} days in total.")

In [ ]:
# Group data by day and sum durations
daily_duration = df_cdr.groupby(df_cdr['date_time'].dt.date)['duration'].sum()

# Plotting
plt.figure(figsize=(12, 6))
daily_duration.plot(kind='bar', color='blue', alpha=0.7)

plt.title('Total Call Duration Over 30 Days')
plt.xlabel('Date')
plt.ylabel('Total Duration in Minutes')

plt.xticks(rotation=45)  # Rotate date labels for better visibility
plt.show()

In [ ]:
# Add a 'day_of_week' column to the DataFrame
df_cdr['day_of_week'] = df_cdr['date_time'].dt.dayofweek

# Group data by 'day_of_week' and sum durations
weekly_duration = df_cdr.groupby('day_of_week')['duration'].sum()

# Map the 'day_of_week' integers to day names for clearer x-axis labels
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
weekly_duration.index = weekly_duration.index.map(day_names)

# Plotting
plt.figure(figsize=(12, 6))
weekly_duration.plot(kind='bar', color='blue', alpha=0.7)

plt.title('Total Call Duration by Day of the Week')
plt.xlabel('Days of the Week')
plt.ylabel('Total Duration in Minutes')

plt.xticks(rotation=45)  # Rotate day labels for better visibility
plt.show()

In [ ]:
# Group data by day and count the number of calls
daily_calls = df_cdr.groupby(df_cdr['date_time'].dt.date).size()

# Plotting
plt.figure(figsize=(12, 6))
daily_calls.plot(kind='bar', color='orange', alpha=0.7)

plt.title('Total Call Counts Over 30 Days')
plt.xlabel('Date')
plt.ylabel('Number of Calls')

plt.xticks(rotation=45)  # Rotate date labels for better visibility
plt.show()

In [ ]:
# Group data by 'day_of_week' and count calls
weekly_calls = df_cdr.groupby('day_of_week').size()

# Map the 'day_of_week' integers to day names for clearer x-axis labels
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
weekly_calls.index = weekly_calls.index.map(day_names)

# Plotting
plt.figure(figsize=(12, 6))
weekly_calls.plot(kind='bar', color='orange', alpha=0.7)

plt.title('Total Call Counts by Day of the Week')
plt.xlabel('Days of the Week')
plt.ylabel('Number of Calls')

plt.xticks(rotation=45)  # Rotate day labels for better visibility
plt.show()

In [ ]:
# Define bins for the duration intervals
bins = [0, 1, 5, 10, 30, 60, 120, 240, 480, float('inf')]  # Adjust the bins as needed
labels = ['0-1 min', '1-5 mins', '5-10 mins', '10-30 mins', '30-60 mins', '1-2 hrs', '2-4 hrs', '4-8 hrs', 'Over 8 hrs']

# Categorize durations into bins
df_cdr['duration_category'] = pd.cut(df_cdr['duration'], bins=bins, labels=labels, right=False)

# Count the number of calls in each duration category
duration_distribution = df_cdr['duration_category'].value_counts().sort_index()

# Plotting
plt.figure(figsize=(12, 6))
bars = duration_distribution.plot(kind='bar', color='blue')
plt.title('Distribution of Call Durations')
plt.xlabel('Duration Intervals')
plt.ylabel('Number of Calls')
plt.xticks(rotation=45)

# Adding text labels above the bars
for bar in bars.patches:
    # Using bar.get_height() to get the height of the bar,
    # and bar.get_x() + bar.get_width() / 2 to calculate center position of the bar
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{int(bar.get_height())}',
             ha='center', va='bottom', color='black', fontsize=9)

plt.show()

In [ ]:
# Filter calls that last less than 60 seconds
short_calls = df_cdr[df_cdr['duration'] < 1]

# Plotting the distribution of call durations under one minute
plt.figure(figsize=(12, 6))
sns.histplot(short_calls['duration'], bins=30, color='blue', kde=True)  # Bins for every 2 seconds
plt.title('Distribution of Call Durations Under One Minute')
plt.xlabel('Duration in Seconds')
plt.ylabel('Number of Calls')
plt.xticks([i/60 for i in range(0, 61, 5)], [f"{i}s" for i in range(0, 61, 5)])  # Setting x-ticks to show seconds
plt.show()

In [ ]:
# Define bins for the duration intervals
bins = [0, 1, 5, 10, 30, 60, 120, 240, 480, float('inf')]  # Adjust the bins as needed
labels = ['0-1 min', '1-5 mins', '5-10 mins', '10-30 mins', '30-60 mins', '1-2 hrs', '2-4 hrs', '4-8 hrs', 'Over 8 hrs']

# Categorize durations into bins
df_cdr['duration_category'] = pd.cut(df_cdr['duration'], bins=bins, labels=labels, right=False)

# Create a box plot
plt.figure(figsize=(12, 6))
sns.boxplot(x='duration_category', y='duration', data=df_cdr, palette='Blues')

# Add title and labels
plt.title('Box Plot of Call Durations')
plt.xlabel('Duration Intervals')
plt.ylabel('Call Duration (minutes)')
plt.xticks(rotation=45)

plt.show()

In [ ]:
# Sum durations per caller_id
total_durations = df_cdr.groupby('caller_id')['duration'].sum()

# Plotting the total durations. Since there are potentially millions of callers,
# we use a histogram and a KDE plot for a better understanding of distribution.
plt.figure(figsize=(12, 6))
sns.histplot(total_durations, bins=100, kde=True, color='green')
plt.title('Distribution of Total Call Duration Per User')
plt.xlabel('Total Duration (minutes)')
plt.ylabel('Frequency')
plt.xscale('log')  # Use logarithmic scale if the range is wide
plt.show()

In [ ]:
# Assuming df_cdr is your DataFrame and it includes a 'caller_id' column
call_counts = df_cdr['caller_id'].value_counts()

# Convert the Series to a DataFrame for easier handling
call_counts_df = call_counts.reset_index()
call_counts_df.columns = ['caller_id', 'number_of_calls']

In [ ]:
# Plotting the distribution of call counts
plt.figure(figsize=(10, 6))
plt.hist(call_counts, bins=100, color='blue', range=(1, call_counts.quantile(0.99)))  # Adjust range and bins as needed
plt.title('Distribution of Call Counts per User')
plt.xlabel('Number of Calls')
plt.ylabel('Frequency')
plt.xscale('log')  # Use logarithmic scale if the data spans several orders of magnitude
# plt.yscale('log')  # Log scale for y-axis can also be helpful in skewed distributions
plt.show()

# Show basic statistics
print(call_counts_df['number_of_calls'].describe())


In [ ]:
# Adding a small constant if there are any zero call counts (unlikely in this case)
call_counts_log = np.log1p(call_counts)

# Plotting the log-transformed data
plt.figure(figsize=(10, 6))
sns.histplot(call_counts_log, bins=50, kde=False, color='green')
plt.title('Log-Transformed Distribution of Call Counts per User')
plt.xlabel('Log of Number of Calls')
plt.ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))  # Setup the plot size
stats.probplot(call_counts_log, dist="norm", plot=ax)  # Generate and plot the Q-Q plot

# Customize the plot
ax.get_lines()[0].set_color('blue')  # Change the color of the points to blue
ax.get_lines()[1].set_color('red')   # Change the color of the line to red
ax.set_title('Q-Q plot of Log-Transformed Call Durations')
ax.set_xlabel('Theoretical Quantiles')
ax.set_ylabel('Ordered Values')

plt.show()  # Display the plot

In [ ]:
# Define the threshold for call counting
lower_threshold = 5
upper_threshold = 200

# Apply thresholds
filtered_call_counts = df_cdr['caller_id'].value_counts()
total_calls_original = df_cdr['caller_id'].value_counts().sum()
filtered_call_counts = filtered_call_counts[(filtered_call_counts >= lower_threshold) & (filtered_call_counts <= upper_threshold)]
filtered_log_counts = np.log1p(filtered_call_counts)

total_calls_filtered = filtered_call_counts.sum()

percentage_filtered = (total_calls_filtered / total_calls_original) * 100

print(f"Percentage of calls remaining after filtering: {percentage_filtered:.2f}%")

# Re-plotting histograms and Q-Q plots
plt.figure(figsize=(12, 6))
sns.histplot(filtered_log_counts, kde=True)
plt.title('Filtered Histogram of Log-Transformed Call Counts')
plt.show()

plt.figure(figsize=(6, 6))
stats.probplot(filtered_log_counts, dist="norm", plot=plt)
plt.show()

# D'Agostino's K^2 test
k2, p = stats.normaltest(filtered_log_counts)
print(f"Statistic: {k2}, p-value: {p}")

# Shapiro-Wilk test
stat, p = stats.shapiro(filtered_log_counts.sample(5000))
print(f"Shapiro-Wilk Test: Statistic={stat}, p-value={p}")

In [ ]:
call_counts = df_cdr['caller_id'].value_counts()

# Define different thresholds for testing
thresholds = range(1, 15)
results = []

for lower_threshold in thresholds:
    # Applying thresholds to filter data
    filtered_call_counts = call_counts[call_counts >= lower_threshold]
    filtered_log_counts = np.log1p(filtered_call_counts)

    # Calculating statistical tests
    k2, p_normaltest = stats.normaltest(filtered_log_counts)
    stat_shapiro, p_shapiro = stats.shapiro(filtered_log_counts.sample(min(5000, len(filtered_log_counts))))  # 样本量过大时需要抽样

    # Storing results
    results.append({
        'lower_threshold': lower_threshold,
        'p_value_normaltest': p_normaltest,
        'p_value_shapiro': p_shapiro
    })

# Converting results to DataFrame for analysis
results_df = pd.DataFrame(results)
print(results_df)

# Plotting p-values against thresholds
plt.figure(figsize=(12, 6))
plt.plot(results_df['lower_threshold'], results_df['p_value_normaltest'], label='D\'Agostino\'s K^2 Test p-value', marker='o')
plt.plot(results_df['lower_threshold'], results_df['p_value_shapiro'], label='Shapiro-Wilk Test p-value', marker='x')
plt.xlabel('Lower Threshold')
plt.ylabel('p-value')
plt.title('Effect of Lower Threshold on Normality Tests')
plt.legend()
plt.grid(True)
plt.show()